# 循环神经网络
:label:`chap_rnn`

到目前为止，我们主要关注的是固定长度的数据。
在介绍 :numref:`chap_regression` 和 :numref:`chap_classification` 中的线性回归和逻辑回归以及 :numref:`chap_perceptrons` 中的多层感知机时，我们很乐意假设每个特征向量 $\mathbf{x}_i$ 由固定数量的组件 $x_1, \dots, x_d$ 组成，其中每个数值特征 $x_j$ 对应一个特定属性。这些数据集有时被称为*表格*数据，因为它们可以排列成表格，每个示例 $i$ 占一行，每个属性占一列。至关重要的是，在表格数据中，我们很少假设列之间有任何特定结构。

随后，在 :numref:`chap_cnn` 中，我们转向了图像数据，输入由图像中每个坐标的原始像素值组成。图像数据很难符合典型的表格数据的标准。在那里，我们需要调用卷积神经网络 (CNNs) 来处理层次结构和不变性。然而，我们的数据仍然是固定长度的。每张 Fashion-MNIST 图像都表示为 $28 \times 28$ 的像素值网格。此外，我们的目标是开发一种模型，该模型只看一张图像然后输出单一预测。但是当我们面对一系列图像（如视频）或需要生成顺序结构的预测（如图像字幕）时该怎么办？

许多学习任务需要处理序列数据。图像字幕、语音合成和音乐生成都需要模型产生由序列组成的输出。在其他领域，如时间序列预测、视频分析和音乐信息检索，模型必须从序列形式的输入中学习。这些需求经常同时出现：例如将一段文本从一种自然语言翻译成另一种自然语言、参与对话或控制机器人等任务，要求模型既摄入又输出有序结构的数据。

循环神经网络 (RNNs) 是深度学习模型，通过*循环*连接来捕捉序列的动态，这可以被看作是节点网络中的循环。乍一看这可能有些反直觉。毕竟，正是前馈神经网络的特性使得计算顺序明确无误。然而，循环边以精确的方式定义，确保不会出现这样的歧义。循环神经网络在时间步（或序列步）上*展开*，并在每个步骤应用*相同*的基础参数。虽然标准连接同步应用于传播每一层的激活到同一时间步的下一层，但循环连接是*动态*的，将信息传递给相邻的时间步。正如 :numref:`fig_unfolded-rnn` 展开视图所示，RNNs 可以被视为前馈神经网络，其中每一层的参数（常规和循环）都在时间步之间共享。

![左边通过循环边描绘了循环连接。右边，我们在时间步上展开了 RNN。这里，循环边跨越相邻的时间步，而常规连接在同一时间步同步计算。](../img/unfolded-rnn.svg)
:label:`fig_unfolded-rnn`

像更广泛的神经网络一样，RNNs 有着跨学科的悠久历史，起源于认知科学家推广的大脑模型，并随后被机器学习社区采纳为实用建模工具。正如我们对深度学习所做的那样，在本书中我们采用机器学习的视角，专注于 RNNs 作为实用工具，由于在诸如手写识别 :cite:`graves2008novel`、机器翻译 :cite:`Sutskever.Vinyals.Le.2014` 和识别医疗诊断 :cite:`Lipton.Kale.2016` 等多样化任务上的突破性成果，RNNs 在 2010 年代变得流行起来。对于有兴趣了解更多背景材料的读者，我们推荐一篇公开可用的全面综述 :cite:`Lipton.Berkowitz.Elkan.2015`。我们还注意到，序列性并不是 RNNs 所独有的。例如，我们已经介绍过的 CNNs 可以适应处理不同长度的数据，例如分辨率不同的图像。此外，RNNs 最近已让出了相当大的市场份额给 Transformer 模型，后者将在 :numref:`chap_attention-and-transformers` 中讨论。然而，RNNs 作为处理深度学习中复杂序列结构的默认模型而崛起，并且至今仍是序列建模的主要模型。RNNs 的故事与序列建模的故事密不可分，这一章既是关于序列建模问题基础知识的一章，也是关于 RNNs 的一章。

一个关键见解为序列建模带来了一场革命。虽然机器学习中的许多基本任务的输入和目标不能轻易表示为固定长度的向量，但它们通常可以表示为可变长度的固定长度向量序列。例如，文档可以表示为词序列；医疗记录通常可以表示为事件序列（就诊、药物、程序、实验室测试、诊断）；视频可以表示为不同长度的静止图像序列。

虽然序列模型出现在许多应用领域，但该领域的基础研究主要是由自然语言处理核心任务的进步推动的。因此，在本章中，我们将重点放在文本数据的阐述和示例上。如果你掌握了这些例子，那么将这些模型应用于其他数据模式应该相对简单。在接下来的几节中，我们将介绍序列的基本符号和一些评估顺序结构模型输出质量的度量方法。之后，我们将讨论语言模型的基本概念，并利用这一讨论来激励我们的第一个 RNN 模型。最后，我们描述了在反向传播通过 RNNs 时计算梯度的方法，并探讨了训练此类网络时常遇到的一些挑战，从而引出 :numref:`chap_modern_rnn` 中将要讨论的现代 RNN 架构。

:begin_tab:toc
 - [sequence](sequence.ipynb)
 - [text-sequence](text-sequence.ipynb)
 - [language-model](language-model.ipynb)
 - [rnn](rnn.ipynb)
 - [rnn-scratch](rnn-scratch.ipynb)
 - [rnn-concise](rnn-concise.ipynb)
 - [bptt](bptt.ipynb)
:end_tab: